# Régression linéaire en PyTorch

In [1]:
import torch

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
xx = torch.rand(3,2)
print(xx)
yy = xx.view(1,-1)
print(yy)
yy.resize_(2,3)
print(yy)

ax = torch.randn(1)
print(ax)
print(ax.size())
ax.requires_grad_(True)
print(ax)

tensor([[0.1481, 0.7299],
        [0.4962, 0.7306],
        [0.0845, 0.9007]])
tensor([[0.1481, 0.7299, 0.4962, 0.7306, 0.0845, 0.9007]])
tensor([[0.1481, 0.7299, 0.4962],
        [0.7306, 0.0845, 0.9007]])
tensor([0.5445])
torch.Size([1])
tensor([0.5445], requires_grad=True)


In [3]:
# Data Generation
np.random.seed(40)
x = np.random.rand(100, 1)
y = 1 + 2 * x + .1 * np.random.randn(100, 1) #  samples from the “standard normal” distribution (mean 0, variance 1)

# Shuffles the indices
idx = np.arange(100)
np.random.shuffle(idx)

# Uses first 80 random indices for train
train_idx = idx[:80]
# Uses the remaining indices for validation
val_idx = idx[80:]

# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]



In [4]:
# Our data was in Numpy arrays, but we need to transform them into PyTorch's Tensors
# and cast them into lower precision
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

# Here we can see the difference - notice that .type() is more useful
# since it also tells us WHERE the tensor is (device)
print(type(x_train), type(x_train_tensor))


x_ndarray = x_train_tensor.numpy()
print(type(x_ndarray))

<class 'numpy.ndarray'> <class 'torch.Tensor'>
<class 'numpy.ndarray'>


In [5]:
torch.manual_seed(42)
# Random initialization of parameters
a = torch.randn(1, requires_grad=True, dtype=torch.float)
b = torch.randn(1, requires_grad=True, dtype=torch.float)

print(a)
print(b)

a.requires_grad_()

tensor([0.3367], requires_grad=True)
tensor([0.1288], requires_grad=True)


tensor([0.3367], requires_grad=True)

In [6]:
from torchviz import make_dot
lr = 1e-1
n_epochs = 1000

torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float)
b = torch.randn(1, requires_grad=True, dtype=torch.float)

for epoch in range(n_epochs):
    yhat = a + b * x_train_tensor
    error = y_train_tensor - yhat
    loss = (error ** 2).mean()
    
    #print(yhat.grad_fn)
    #print(error.grad_fn)
    #print(loss.grad_fn)

    # No more manual computation of gradients! 
    # a_grad = -2 * error.mean()
    # b_grad = -2 * (x_tensor * error).mean()
    
    # We just tell PyTorch to work its way BACKWARDS from the specified loss!
    loss.backward()
 
    # Let's check the computed gradients...
    if epoch % 100 == 0:
        print(a.grad)
        print(a)
        print(b.grad)
    
    # What about UPDATING the parameters? 
    
    # FIRST ATTEMPT
    # AttributeError: 'NoneType' object has no attribute 'zero_'
    #print("dMSE/da avant ",a.grad)
    #a = a - lr * a.grad
    #b = b - lr * b.grad
    #print("dMSE/da apres ",a.grad)

    #make_dot(yhat).render("attached", format="png")

    # SECOND ATTEMPT
    # RuntimeError: a leaf Variable that requires grad has been used in an in-place operation.
    #a -= lr * a.grad
    #b -= lr * b.grad        
    
    # THIRD ATTEMPT
    # Use NO_GRAD to keep the update out of the gradient computation
    with torch.no_grad():
        a -= lr * a.grad
        b -= lr * b.grad
    
    # PyTorch is "clingy" to its computed gradients, we need to tell it to let it go...
    a.grad.zero_()
    b.grad.zero_()

print(a, b)

tensor([-3.1019])
tensor([0.3367], requires_grad=True)
tensor([-1.8132])
tensor([0.0211])
tensor([1.1555], requires_grad=True)
tensor([-0.0406])
tensor([0.0054])
tensor([1.0396], requires_grad=True)
tensor([-0.0104])
tensor([0.0014])
tensor([1.0099], requires_grad=True)
tensor([-0.0027])
tensor([0.0004])
tensor([1.0024], requires_grad=True)
tensor([-0.0007])
tensor([9.0291e-05])
tensor([1.0004], requires_grad=True)
tensor([-0.0002])
tensor([2.3011e-05])
tensor([0.9999], requires_grad=True)
tensor([-4.4408e-05])
tensor([6.0038e-06])
tensor([0.9998], requires_grad=True)
tensor([-1.1315e-05])
tensor([1.5656e-06])
tensor([0.9998], requires_grad=True)
tensor([-2.9360e-06])
tensor([5.6135e-07])
tensor([0.9998], requires_grad=True)
tensor([-6.8033e-07])
tensor([0.9998], requires_grad=True) tensor([1.9619], requires_grad=True)


In [7]:
from torch import optim
from torch import nn

lr = 1e-1
n_epochs = 1000

torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float)
b = torch.randn(1, requires_grad=True, dtype=torch.float)
print(a, b)

# Defines a MSE loss function
loss_fn = nn.MSELoss(reduction='mean')
# Defines a SGD optimizer to update the parameters
optimizer = optim.SGD([a, b], lr=lr)

for epoch in range(n_epochs):
    yhat = a + b * x_train_tensor
    
    # No more manual loss!
    # error = y_tensor - yhat
    # loss = (error ** 2).mean()
    loss = loss_fn(y_train_tensor, yhat)
 
    loss.backward()    
    
    # No more manual update!
    # with torch.no_grad():
    #     a -= lr * a.grad
    #     b -= lr * b.grad
    optimizer.step()
    
    # No more telling PyTorch to let gradients go!
    # a.grad.zero_()
    # b.grad.zero_()
    optimizer.zero_grad()
    
print(a, b)

tensor([0.3367], requires_grad=True) tensor([0.1288], requires_grad=True)
tensor([0.9998], requires_grad=True) tensor([1.9619], requires_grad=True)


In [8]:
class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # To make "a" and "b" real parameters of the model, we need to wrap them with nn.Parameter
        self.a = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        
    def forward(self, x):
        # Computes the outputs / predictions
        return self.a + self.b * x

In [9]:
class LayerLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # Instead of our custom parameters, we use a Linear layer with single input and single output
        self.linear = nn.Linear(1, 1)
                
    def forward(self, x):
        # Now it only takes a call to the layer to make predictions
        return self.linear(x)

In [10]:
torch.manual_seed(42)

# Now we can create a model 
#model = ManualLinearRegression()
model = LayerLinearRegression()
# We can also inspect its parameters using its state_dict
print(model.state_dict())

lr = 1e-1
n_epochs = 1000

loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(n_epochs):
    model.train()

    # No more manual prediction!
    # yhat = a + b * x_tensor
    yhat = model(x_train_tensor)
    
    loss = loss_fn(y_train_tensor, yhat)
    loss.backward()    
    optimizer.step()
    optimizer.zero_grad()
    
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[0.7645]])), ('linear.bias', tensor([0.8300]))])
OrderedDict([('linear.weight', tensor([[1.9619]])), ('linear.bias', tensor([0.9998]))])
